In [2]:
!pip install wikipedia
import pandas as pd
import numpy as np


import re
import wikipedia
from bs4 import BeautifulSoup
import urllib.request



  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=a1f3b6db8d51b44b443f27d70acfd80d1f9c9f99e906b8a571749b147848436a
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
% cd '/content/drive/My Drive/Colab Notebooks/Omdena'

/content/drive/My Drive/Colab Notebooks/Omdena


In [0]:
df = pd.read_csv("ibtracs_tcedat_emdat_inner_without_NaN_Name.csv")

In [167]:
df.shape

(984, 83)

In [168]:
df.columns

Index(['Unnamed: 0', 'SID', 'ISO', 'NAME', 'BASIN', 'SUBBASIN', 'NATURE',
       'TRACK_TYPE', 'ISO_TIME', 'COORDS', 'TOTAL_HRS', 'DAY_HRS', 'NIGHT_HRS',
       'USA_SSHS', 'YEAR_min', 'YEAR_max', 'WMO_WIND_min', 'WMO_WIND_max',
       'WMO_WIND_mean', 'WIND_CALC_min', 'WIND_CALC_max', 'WIND_CALC_mean',
       'WMO_PRES_min', 'WMO_PRES_max', 'WMO_PRES_mean', 'PRES_CALC_min',
       'PRES_CALC_max', 'PRES_CALC_mean', 'STORM_SPD_min', 'STORM_SPD_max',
       'STORM_SPD_mean', 'STORM_DR_min', 'STORM_DR_max', 'STORM_DR_mean',
       'year', 'IBTrACS_ID', 'TC_name', 'NatCatSERVICE_ID', 'genesis_basin',
       'countries_affected', 'ISO3', 'v_land_SI', 'v_land_kn', '34kn_pop',
       '34kn_assets', '64kn_pop', '64kn_assets', '96kn_pop', '96kn_assets',
       'ibtracs_name', 'merged_name', 'CPI', 'Continent', 'Country name',
       'End day', 'End month', 'End year', 'Entry criteria', 'Group',
       'Insured losses ('000$)', 'Local time', 'Location', 'Magnitude (value)',
       'Num affected

Columns to work with: 'Num affected', 'Num homeless', 'Num injured'

In [169]:
df['Num affected'].isnull().sum()

305

In [170]:
df['Num homeless'].isnull().sum()

757

In [171]:
df['Num injured'].isnull().sum()

582

In [0]:
new_df = pd.DataFrame(columns=df.columns)

In [0]:
cyclone_list = []
for index, row in df.iterrows():
    if pd.isnull(row['Num affected']) or pd.isnull(row['Num homeless']) or pd.isnull(row['Num injured']):
        a_series = pd.Series(row, index=df.columns)
        new_df = new_df.append(row, ignore_index=True)
        cyclone_list.append([row['NAME'], row['Start year']])

        

In [174]:
new_df

,Unnamed: 0,SID,ISO,NAME,BASIN,SUBBASIN,NATURE,TRACK_TYPE,ISO_TIME,COORDS,TOTAL_HRS,DAY_HRS,NIGHT_HRS,USA_SSHS,YEAR_min,YEAR_max,WMO_WIND_min,WMO_WIND_max,WMO_WIND_mean,WIND_CALC_min,WIND_CALC_max,WIND_CALC_mean,WMO_PRES_min,WMO_PRES_max,WMO_PRES_mean,PRES_CALC_min,PRES_CALC_max,PRES_CALC_mean,STORM_SPD_min,STORM_SPD_max,STORM_SPD_mean,STORM_DR_min,STORM_DR_max,STORM_DR_mean,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,...,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets,ibtracs_name,merged_name,CPI,Continent,Country name,End day,End month,End year,Entry criteria,Group,Insured losses ('000$),Local time,Location,Magnitude (value),Num affected,Num homeless,Num injured,Origin,Reconstruction cost ('000$),Region,River basin,Seq,Start day,Start month,Start year,Subgroup,Subsubtype,Subtype,Total affected,Total damage ('000$),Total deaths,Type,Unnamed: 0.1,multi-name
0,0,1949163N07145,JPN,DELLA,WP,MM,ET,main,"['1949-06-20 15:00:00', '1949-06-20 18:00:00',...","[[31.99, 130.68], [33.52, 130.97], [41.0, 140....",12.0,0.0,12.0,-1.0,1949.0,1949.0,NaN,NaN,NaN,23.0,60.2,40.900000,NaN,NaN,NaN,967.0,1003.0,986.375000,12.0,33.0,21.500000,7.0,75.0,40.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.327572,Asia,Japan,NaN,6.0,1949,Kill,Natural,NaN,NaN,Western Japan,NaN,194046.0,NaN,NaN,NaN,NaN,Eastern Asia,NaN,7,NaN,6.0,1949,Meteorological,NaN,Tropical cyclone,194046.0,NaN,419.0,Storm,494,0.0
1,1,1949313N06137,PHL,RENA,WP,MM,TS,main,"['1949-11-11 09:00:00', '1949-11-11 15:00:00',...","[[9.49, 125.55], [9.99, 124.0], [10.3, 123.13]...",12.0,11.0,1.0,0.0,1949.0,1949.0,NaN,NaN,NaN,56.0,57.6,56.950000,NaN,NaN,NaN,992.0,993.0,992.750000,15.0,19.0,17.750000,289.0,299.0,292.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.327572,Asia,Philippines (the),31.0,10.0,1949,Kill,Natural,NaN,NaN,"Cebu City, Negros, Panay",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,14,31.0,10.0,1949,Meteorological,NaN,Tropical cyclone,NaN,NaN,1000.0,Storm,1401,0.0
2,2,1950241N23140,JPN,JANE,WP,MM,TS,main,"['1950-09-03 03:00:00', '1950-09-03 06:00:00',...","[[34.94, 135.48], [36.1, 136.09], [37.27, 136....",21.0,0.0,21.0,0.0,1950.0,1950.0,NaN,NaN,NaN,40.0,77.6,60.900000,NaN,NaN,NaN,961.5,993.0,973.500000,25.0,29.0,27.000000,21.0,46.0,31.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.431937,Asia,Japan,NaN,9.0,1950,Kill,Natural,NaN,NaN,Entire nation except Kyushu,NaN,642117.0,NaN,NaN,NaN,NaN,Eastern Asia,NaN,10,NaN,9.0,1950,Meteorological,NaN,Tropical cyclone,642117.0,NaN,509.0,Storm,512,0.0
3,3,1951119N06145,PHL,IRIS,WP,MM,TS,main,"['1951-05-04 21:00:00', '1951-05-05 00:00:00',...","[[11.99, 125.07], [12.12, 124.72], [13.73, 121...",6.0,0.0,6.0,0.0,1951.0,1951.0,NaN,NaN,NaN,56.2,103.4,77.550000,930.0,975.0,952.5,925.0,974.0,949.833333,8.0,10.0,9.000000,287.0,303.0,295.000000,1951.0,1951119N06145,IRIS,NaN,WP,multi,...,57147773.0,5.206064e+11,28028984.0,2.366082e+11,1319917.0,1.283009e+10,NaN,IRIS,10.175534,Asia,Philippines (the),24.0,4.0,1951,Kill,Natural,NaN,NaN,"Samar, South Luzon",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,11,24.0,4.0,1951,Meteorological,NaN,Tropical cyclone,NaN,NaN,156.0,Storm,1419,0.0
4,4,1951224N12316,JAM,CHARLIE,NaN,CS,TS,main,"['1951-08-18 03:00:00', '1951-08-18 06:00:00']","[[17.9, -76.9], [18.1, -77.8]]",3.0,0.0,3.0,2.0,1951.0,1951.0,85.0,110.0,97.5,85.0,110.0,97.500000,NaN,NaN,NaN,974.0,975.0,974.500000,16.0,17.0,16.500000,281.0,288.0,284.500000,1951.0,1951224N12316,CHARLIE,NaN,NaN,multi,...,2788659.0,2.148198e+10,2788659.0,2.148198e+10,2552903.0,1.948346e+10,NaN,CHARLIE,10.175534,Americas,Jamaica,18.0,8.0,1951,Kill,Natural,NaN,NaN,South coast,NaN,20000.0,NaN,200.0,NaN,NaN,Caribbean,NaN,16,17.0,8.0,1951,Meteorological,NaN,Tropical cyclone,20200.0,56000.0,154.0,Storm,529,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [175]:
df_homeless = pd.DataFrame(columns=df.columns)
list_of_homeless = []

for index, row in df.iterrows():
    if pd.isnull(row['Num homeless']):
        a_series = pd.Series(row, index=df.columns)
        df_homeless = df_homeless.append(row, ignore_index=True)
        list_of_homeless.append(row['NAME'])
        
df_homeless

,Unnamed: 0,SID,ISO,NAME,BASIN,SUBBASIN,NATURE,TRACK_TYPE,ISO_TIME,COORDS,TOTAL_HRS,DAY_HRS,NIGHT_HRS,USA_SSHS,YEAR_min,YEAR_max,WMO_WIND_min,WMO_WIND_max,WMO_WIND_mean,WIND_CALC_min,WIND_CALC_max,WIND_CALC_mean,WMO_PRES_min,WMO_PRES_max,WMO_PRES_mean,PRES_CALC_min,PRES_CALC_max,PRES_CALC_mean,STORM_SPD_min,STORM_SPD_max,STORM_SPD_mean,STORM_DR_min,STORM_DR_max,STORM_DR_mean,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,...,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets,ibtracs_name,merged_name,CPI,Continent,Country name,End day,End month,End year,Entry criteria,Group,Insured losses ('000$),Local time,Location,Magnitude (value),Num affected,Num homeless,Num injured,Origin,Reconstruction cost ('000$),Region,River basin,Seq,Start day,Start month,Start year,Subgroup,Subsubtype,Subtype,Total affected,Total damage ('000$),Total deaths,Type,Unnamed: 0.1,multi-name
0,0,1949163N07145,JPN,DELLA,WP,MM,ET,main,"['1949-06-20 15:00:00', '1949-06-20 18:00:00',...","[[31.99, 130.68], [33.52, 130.97], [41.0, 140....",12.0,0.0,12.0,-1.0,1949.0,1949.0,NaN,NaN,NaN,23.0,60.2,40.900000,NaN,NaN,NaN,967.0,1003.0,986.375000,12.0,33.0,21.500000,7.0,75.0,40.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.327572,Asia,Japan,NaN,6.0,1949,Kill,Natural,NaN,NaN,Western Japan,NaN,194046.0,NaN,NaN,NaN,NaN,Eastern Asia,NaN,7,NaN,6.0,1949,Meteorological,NaN,Tropical cyclone,194046.0,NaN,419.0,Storm,494,0.0
1,1,1949313N06137,PHL,RENA,WP,MM,TS,main,"['1949-11-11 09:00:00', '1949-11-11 15:00:00',...","[[9.49, 125.55], [9.99, 124.0], [10.3, 123.13]...",12.0,11.0,1.0,0.0,1949.0,1949.0,NaN,NaN,NaN,56.0,57.6,56.950000,NaN,NaN,NaN,992.0,993.0,992.750000,15.0,19.0,17.750000,289.0,299.0,292.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.327572,Asia,Philippines (the),31.0,10.0,1949,Kill,Natural,NaN,NaN,"Cebu City, Negros, Panay",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,14,31.0,10.0,1949,Meteorological,NaN,Tropical cyclone,NaN,NaN,1000.0,Storm,1401,0.0
2,2,1950241N23140,JPN,JANE,WP,MM,TS,main,"['1950-09-03 03:00:00', '1950-09-03 06:00:00',...","[[34.94, 135.48], [36.1, 136.09], [37.27, 136....",21.0,0.0,21.0,0.0,1950.0,1950.0,NaN,NaN,NaN,40.0,77.6,60.900000,NaN,NaN,NaN,961.5,993.0,973.500000,25.0,29.0,27.000000,21.0,46.0,31.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.431937,Asia,Japan,NaN,9.0,1950,Kill,Natural,NaN,NaN,Entire nation except Kyushu,NaN,642117.0,NaN,NaN,NaN,NaN,Eastern Asia,NaN,10,NaN,9.0,1950,Meteorological,NaN,Tropical cyclone,642117.0,NaN,509.0,Storm,512,0.0
3,3,1951119N06145,PHL,IRIS,WP,MM,TS,main,"['1951-05-04 21:00:00', '1951-05-05 00:00:00',...","[[11.99, 125.07], [12.12, 124.72], [13.73, 121...",6.0,0.0,6.0,0.0,1951.0,1951.0,NaN,NaN,NaN,56.2,103.4,77.550000,930.0,975.0,952.5,925.0,974.0,949.833333,8.0,10.0,9.000000,287.0,303.0,295.000000,1951.0,1951119N06145,IRIS,NaN,WP,multi,...,57147773.0,5.206064e+11,28028984.0,2.366082e+11,1319917.0,1.283009e+10,NaN,IRIS,10.175534,Asia,Philippines (the),24.0,4.0,1951,Kill,Natural,NaN,NaN,"Samar, South Luzon",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,11,24.0,4.0,1951,Meteorological,NaN,Tropical cyclone,NaN,NaN,156.0,Storm,1419,0.0
4,4,1951224N12316,JAM,CHARLIE,NaN,CS,TS,main,"['1951-08-18 03:00:00', '1951-08-18 06:00:00']","[[17.9, -76.9], [18.1, -77.8]]",3.0,0.0,3.0,2.0,1951.0,1951.0,85.0,110.0,97.5,85.0,110.0,97.500000,NaN,NaN,NaN,974.0,975.0,974.500000,16.0,17.0,16.500000,281.0,288.0,284.500000,1951.0,1951224N12316,CHARLIE,NaN,NaN,multi,...,2788659.0,2.148198e+10,2788659.0,2.148198e+10,2552903.0,1.948346e+10,NaN,CHARLIE,10.175534,Americas,Jamaica,18.0,8.0,1951,Kill,Natural,NaN,NaN,South coast,NaN,20000.0,NaN,200.0,NaN,NaN,Caribbean,NaN,16,17.0,8.0,1951,Meteorological,NaN,Tropical cyclone,20200.0,56000.0,154.0,Storm,529,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [176]:
df_injured = pd.DataFrame(columns=df.columns)

for index, row in df.iterrows():
    if pd.isnull(row['Num injured']):
        a_series = pd.Series(row, index=df.columns)
        df_injured = df_injured.append(row, ignore_index=True)
        
df_injured

,Unnamed: 0,SID,ISO,NAME,BASIN,SUBBASIN,NATURE,TRACK_TYPE,ISO_TIME,COORDS,TOTAL_HRS,DAY_HRS,NIGHT_HRS,USA_SSHS,YEAR_min,YEAR_max,WMO_WIND_min,WMO_WIND_max,WMO_WIND_mean,WIND_CALC_min,WIND_CALC_max,WIND_CALC_mean,WMO_PRES_min,WMO_PRES_max,WMO_PRES_mean,PRES_CALC_min,PRES_CALC_max,PRES_CALC_mean,STORM_SPD_min,STORM_SPD_max,STORM_SPD_mean,STORM_DR_min,STORM_DR_max,STORM_DR_mean,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,...,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets,ibtracs_name,merged_name,CPI,Continent,Country name,End day,End month,End year,Entry criteria,Group,Insured losses ('000$),Local time,Location,Magnitude (value),Num affected,Num homeless,Num injured,Origin,Reconstruction cost ('000$),Region,River basin,Seq,Start day,Start month,Start year,Subgroup,Subsubtype,Subtype,Total affected,Total damage ('000$),Total deaths,Type,Unnamed: 0.1,multi-name
0,0,1949163N07145,JPN,DELLA,WP,MM,ET,main,"['1949-06-20 15:00:00', '1949-06-20 18:00:00',...","[[31.99, 130.68], [33.52, 130.97], [41.0, 140....",12.0,0.0,12.0,-1.0,1949.0,1949.0,NaN,NaN,NaN,23.000000,60.200000,40.900000,NaN,NaN,NaN,967.0,1003.0,986.375000,12.0,33.0,21.500000,7.0,75.0,40.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.327572,Asia,Japan,NaN,6.0,1949,Kill,Natural,NaN,NaN,Western Japan,NaN,194046.0,NaN,NaN,NaN,NaN,Eastern Asia,NaN,7,NaN,6.0,1949,Meteorological,NaN,Tropical cyclone,194046.0,NaN,419.0,Storm,494,0.0
1,1,1949313N06137,PHL,RENA,WP,MM,TS,main,"['1949-11-11 09:00:00', '1949-11-11 15:00:00',...","[[9.49, 125.55], [9.99, 124.0], [10.3, 123.13]...",12.0,11.0,1.0,0.0,1949.0,1949.0,NaN,NaN,NaN,56.000000,57.600000,56.950000,NaN,NaN,NaN,992.0,993.0,992.750000,15.0,19.0,17.750000,289.0,299.0,292.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.327572,Asia,Philippines (the),31.0,10.0,1949,Kill,Natural,NaN,NaN,"Cebu City, Negros, Panay",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,14,31.0,10.0,1949,Meteorological,NaN,Tropical cyclone,NaN,NaN,1000.0,Storm,1401,0.0
2,2,1950241N23140,JPN,JANE,WP,MM,TS,main,"['1950-09-03 03:00:00', '1950-09-03 06:00:00',...","[[34.94, 135.48], [36.1, 136.09], [37.27, 136....",21.0,0.0,21.0,0.0,1950.0,1950.0,NaN,NaN,NaN,40.000000,77.600000,60.900000,NaN,NaN,NaN,961.5,993.0,973.500000,25.0,29.0,27.000000,21.0,46.0,31.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.431937,Asia,Japan,NaN,9.0,1950,Kill,Natural,NaN,NaN,Entire nation except Kyushu,NaN,642117.0,NaN,NaN,NaN,NaN,Eastern Asia,NaN,10,NaN,9.0,1950,Meteorological,NaN,Tropical cyclone,642117.0,NaN,509.0,Storm,512,0.0
3,3,1951119N06145,PHL,IRIS,WP,MM,TS,main,"['1951-05-04 21:00:00', '1951-05-05 00:00:00',...","[[11.99, 125.07], [12.12, 124.72], [13.73, 121...",6.0,0.0,6.0,0.0,1951.0,1951.0,NaN,NaN,NaN,56.200000,103.400000,77.550000,930.0,975.0,952.50,925.0,974.0,949.833333,8.0,10.0,9.000000,287.0,303.0,295.000000,1951.0,1951119N06145,IRIS,NaN,WP,multi,...,57147773.0,5.206064e+11,28028984.0,2.366082e+11,1319917.0,1.283009e+10,NaN,IRIS,10.175534,Asia,Philippines (the),24.0,4.0,1951,Kill,Natural,NaN,NaN,"Samar, South Luzon",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,11,24.0,4.0,1951,Meteorological,NaN,Tropical cyclone,NaN,NaN,156.0,Storm,1419,0.0
4,5,1951281N11147,JPN,RUTH,WP,MM,TS,main,"['1951-10-14 12:00:00', '1951-10-14 18:00:00',...","[[32.6, 131.22], [34.85, 133.62], [36.75, 137....",18.0,0.0,18.0,0.0,1951.0,1951.0,NaN,NaN,NaN,52.666667,85.333333,64.933333,952.0,978.0,967.75,952.0,976.5,968.700000,29.0,33.0,31.400000,37.0,69.0,56.800000,1951.0,1951281N11147,RUTH,NaN,WP,single,...,96950843.0,1.145456e+13,33303723.0,4.025185e+12,2310209.0,2.805463e+11,NaN,RUTH,10.175534,Asia,Japan,NaN,10.0,1951,Kill,Natural,NaN,NaN,Entire nation except Hokkaido,NaN,NaN,NaN,NaN,NaN,NaN,Eastern Asia,NaN,19,NaN,10.0,1951,Meteorological,NaN,Tropical cyclone,NaN,NaN,943.0,Storm,530,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [177]:
df_affected = pd.DataFrame(columns=df.columns)

for index, row in df.iterrows():
    if pd.isnull(row['Num affected']):
        a_series = pd.Series(row, index=df.columns)
        df_affected = df_affected.append(row, ignore_index=True)
        
df_affected

,Unnamed: 0,SID,ISO,NAME,BASIN,SUBBASIN,NATURE,TRACK_TYPE,ISO_TIME,COORDS,TOTAL_HRS,DAY_HRS,NIGHT_HRS,USA_SSHS,YEAR_min,YEAR_max,WMO_WIND_min,WMO_WIND_max,WMO_WIND_mean,WIND_CALC_min,WIND_CALC_max,WIND_CALC_mean,WMO_PRES_min,WMO_PRES_max,WMO_PRES_mean,PRES_CALC_min,PRES_CALC_max,PRES_CALC_mean,STORM_SPD_min,STORM_SPD_max,STORM_SPD_mean,STORM_DR_min,STORM_DR_max,STORM_DR_mean,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,...,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets,ibtracs_name,merged_name,CPI,Continent,Country name,End day,End month,End year,Entry criteria,Group,Insured losses ('000$),Local time,Location,Magnitude (value),Num affected,Num homeless,Num injured,Origin,Reconstruction cost ('000$),Region,River basin,Seq,Start day,Start month,Start year,Subgroup,Subsubtype,Subtype,Total affected,Total damage ('000$),Total deaths,Type,Unnamed: 0.1,multi-name
0,1,1949313N06137,PHL,RENA,WP,MM,TS,main,"['1949-11-11 09:00:00', '1949-11-11 15:00:00',...","[[9.49, 125.55], [9.99, 124.0], [10.3, 123.13]...",12.0,11.0,1.0,0.0,1949.0,1949.0,NaN,NaN,NaN,56.000000,57.600000,56.950000,NaN,NaN,NaN,992.000000,993.000000,992.750000,15.0,19.0,17.750000,289.0,299.0,292.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.327572,Asia,Philippines (the),31.0,10.0,1949,Kill,Natural,NaN,NaN,"Cebu City, Negros, Panay",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,14,31.0,10.0,1949,Meteorological,NaN,Tropical cyclone,NaN,NaN,1000.0,Storm,1401,0.0
1,3,1951119N06145,PHL,IRIS,WP,MM,TS,main,"['1951-05-04 21:00:00', '1951-05-05 00:00:00',...","[[11.99, 125.07], [12.12, 124.72], [13.73, 121...",6.0,0.0,6.0,0.0,1951.0,1951.0,NaN,NaN,NaN,56.200000,103.400000,77.550000,930.0,975.0,952.500000,925.000000,974.000000,949.833333,8.0,10.0,9.000000,287.0,303.0,295.000000,1951.0,1951119N06145,IRIS,NaN,WP,multi,...,57147773.0,5.206064e+11,28028984.0,2.366082e+11,1319917.0,1.283009e+10,NaN,IRIS,10.175534,Asia,Philippines (the),24.0,4.0,1951,Kill,Natural,NaN,NaN,"Samar, South Luzon",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,11,24.0,4.0,1951,Meteorological,NaN,Tropical cyclone,NaN,NaN,156.0,Storm,1419,0.0
2,5,1951281N11147,JPN,RUTH,WP,MM,TS,main,"['1951-10-14 12:00:00', '1951-10-14 18:00:00',...","[[32.6, 131.22], [34.85, 133.62], [36.75, 137....",18.0,0.0,18.0,0.0,1951.0,1951.0,NaN,NaN,NaN,52.666667,85.333333,64.933333,952.0,978.0,967.750000,952.000000,976.500000,968.700000,29.0,33.0,31.400000,37.0,69.0,56.800000,1951.0,1951281N11147,RUTH,NaN,WP,single,...,96950843.0,1.145456e+13,33303723.0,4.025185e+12,2310209.0,2.805463e+11,NaN,RUTH,10.175534,Asia,Japan,NaN,10.0,1951,Kill,Natural,NaN,NaN,Entire nation except Hokkaido,NaN,NaN,NaN,NaN,NaN,NaN,Eastern Asia,NaN,19,NaN,10.0,1951,Meteorological,NaN,Tropical cyclone,NaN,NaN,943.0,Storm,530,0.0
3,6,1951321N09143,PHL,WANDA,WP,MM,TS,main,"['1951-11-20 12:00:00', '1951-11-21 00:00:00',...","[[11.43, 125.35], [12.43, 123.27], [12.7, 121....",3.0,3.0,0.0,2.0,1951.0,1951.0,NaN,NaN,NaN,75.400000,85.600000,80.050000,985.0,990.0,988.333333,978.666667,984.333333,982.333333,9.0,12.0,10.250000,276.0,298.0,285.000000,1951.0,1951321N09143,WANDA,NaN,WP,multi,...,27831628.0,2.808894e+11,7954723.0,8.283046e+10,1826023.0,2.091981e+10,NaN,WANDA,10.175534,Asia,Philippines (the),15.0,11.0,1951,Kill,Natural,NaN,NaN,"Visayas, Bicol",NaN,NaN,NaN,NaN,NaN,NaN,South-Eastern Asia,NaN,21,15.0,11.0,1951,Meteorological,NaN,Tropical cyclone,NaN,NaN,87.0,Storm,1420,0.0
4,8,1952180N05144,PHL,EMMA,WP,MM,TS,main,"['1952-07-02 12:00:00', '1952-07-02 15:00:00',...","[[10.47, 123.87], [10.65, 123.24], [10.83, 122...",9.0,8.0,1.0,0.0,1952.0,1952.0,NaN,NaN,NaN,59.000000,67.400000,63.500000,980.0,983.0,981.500000,981.666667,985.333333,983.500000,11.0,13.0,12.250000,287.0,291.0,288.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.410354,Asia,Philippines (the),28.0,6.0,1952,Kill,Natural,NaN,NaN,Central,NaN,NaN,NaN,103.0,NaN,NaN,South-Eastern Asia,NaN,3,28.0,6.0,1952,Meteorological,Na

In [0]:
list_of_cyclone = []
list_of_tropical_storm = []
list_of_typhoon = []
list_of_cyclone_with_year = []
list_of_tropical_storm_with_year = []
list_of_typhoon_with_year = []
list_of_hurricane = []
list_of_hurricane_with_year = []
original_cyclone_list = cyclone_list.copy()

In [0]:
for cyclone in cyclone_list:
  try:
    cyclone_name = cyclone[0].capitalize()
    url = "https://en.wikipedia.org/wiki/Cyclone_" + cyclone_name
    page = urllib.request.urlopen(url)
    list_of_cyclone.append(cyclone)
    cyclone_list.remove(cyclone)
  except:
    try:
      cyclone_name = cyclone[0].capitalize()
      url = "https://en.wikipedia.org/wiki/Tropical_Storm_" + cyclone_name
      page = urllib.request.urlopen(url)
      list_of_tropical_storm.append(cyclone)
      cyclone_list.remove(cyclone)
    except:
      try:
        cyclone_name = cyclone[0].capitalize()
        url = "https://en.wikipedia.org/wiki/Typhoon_" + cyclone_name
        page = urllib.request.urlopen(url)
        list_of_typhoon.append(cyclone)
        cyclone_list.remove(cyclone)
      except:
        try:
          cyclone_name = cyclone[0].capitalize()
          url = "https://en.wikipedia.org/wiki/Hurricane_" + cyclone_name
          page = urllib.request.urlopen(url)
          list_of_hurricane.append(cyclone)
          cyclone_list.remove(cyclone)
        except:
          try:
            cyclone_name = cyclone[0].capitalize()
            url = "https://en.wikipedia.org/wiki/Hurricane_" + cyclone_name + "_(" + str(cyclone[1]) + ")"
            page = urllib.request.urlopen(url)
            list_of_hurricane_with_year.append(cyclone)
            cyclone_list.remove(cyclone)
          except:
            try:
              cyclone_name = cyclone[0].capitalize()
              url = "https://en.wikipedia.org/wiki/Tropical_Storm_" + cyclone_name + "_(" + str(cyclone[1]) + ")"
              page = urllib.request.urlopen(url)
              list_of_tropical_storm_with_year.append(cyclone)
              cyclone_list.remove(cyclone)
            except:
              try:
                cyclone_name = cyclone[0].capitalize()
                url = "https://en.wikipedia.org/wiki/Typhoon_" + cyclone_name + "_(" + str(cyclone[1]) + ")"
                page = urllib.request.urlopen(url)
                list_of_typhoon_with_year.append(cyclone)
                cyclone_list.remove(cyclone)
              except:
                try:
                  cyclone_name = cyclone[0].capitalize()
                  url = "https://en.wikipedia.org/wiki/Cyclone_" + cyclone_name + "_(" + str(cyclone[1]) + ")"
                  page = urllib.request.urlopen(url)
                  list_of_cyclone_with_year.append(cyclone)
                  cyclone_list.remove(cyclone)
                except:
                  pass


   

In [200]:
len(cyclone_list)

33

In [162]:
len(list_of_cyclone), len(list_of_cyclone_with_year), len(list_of_hurricane), len(list_of_hurricane_with_year), len(list_of_tropical_storm), len(list_of_tropical_storm_with_year), len(list_of_typhoon), len(list_of_typhoon_with_year)

(178, 58, 43, 9, 296, 23, 176, 71)

In [163]:
cyclone_list

[['DELLA', 1949],
 ['BEATRICE', 1959],
 ['BEATRICE', 1959],
 ['COLLEEN', 1969],
 ['ELSIE', 1969],
 ['INES', 1975],
 ['TED', 1976],
 ['EMILIE', 1977],
 ['ANGELE', 1978],
 ['ANGELE', 1978],
 ['ANGELE', 1978],
 ['ANGELE', 1978],
 ['ENID', 1980],
 ['ELECTRE', 1982],
 ['FRIDA', 1982],
 ['JUSTINE', 1982],
 ['DOMINIC', 1982],
 ['ELLIS', 1982],
 ['GEORGIA', 1983],
 ['DOM', 1986],
 ['GEORGIA', 1986],
 ['AIVU', 1989],
 ['GARY', 1992],
 ['FLO', 1993],
 ['FRANKIE', 1996],
 ['FRITZ', 1997],
 ['DOMINIC', 2009],
 ['SON-TINH', 2012],
 ['SON-TINH', 2012],
 ['MAHASEN:VIYARU', 2013],
 ['ELIAKIM', 2018],
 ['SON-TINH', 2018],
 ['SON-TINH', 2018]]

In [123]:
'SON-TINH'.capitalize()

'Son-tinh'